In [1]:
import sys
sys.path.append('../')
from utils import *
from model import *

In [2]:
data_dir = '/datadrive/data_med/'
batch_size = 256
batch_num  = 128
epoch_num  = 200
samp_num   = 7

device = torch.device("cuda:3")
graph = dill.load(open(data_dir + 'graph.pk', 'rb'))

In [3]:
train_range = {t: True for t in graph.times if t != None and t < 2015}
valid_range = {t: True for t in graph.times if t != None and t >= 2015  and t <= 2016}
test_range  = {t: True for t in graph.times if t != None and t > 2016}

In [4]:
def pf_sample(seed, papers, pairs, t_range, batch_size, test = False):
    np.random.seed(seed)
    _time = np.random.choice(list(papers.keys()))
    sampn = min(len(papers[_time]), batch_size)
    pids = np.array(papers[_time])[np.random.choice(len(papers[_time]), sampn, replace = False)]
    fids = []
    edge = defaultdict(lambda: {})
    for x_id, p_id in enumerate(pids):
        f_ids = pairs[p_id]
        for f_id in f_ids:
            if f_id not in fids:
                fids += [f_id]
            edge[x_id][fids.index(f_id)] = True
    pids = np.stack([pids, np.repeat([_time], sampn)]).T
    fids = np.stack([fids, np.repeat([_time], len(fids))]).T
 
    feature, times, edge_list, _ = sample_subgraph(graph, t_range, \
                inp = {'paper': pids, 'field': fids}, sampled_depth = 3, sampled_number = 100)

    el = []
    for i in edge_list['paper']['field']['rev_PF_in_L1']:
        if i[0] in edge and i[1] in edge[i[0]]:
            continue
        el += [i]
    edge_list['paper']['field']['rev_PF_in_L1'] = el

    el = []
    for i in edge_list['field']['paper']['PF_in_L1']:
        if i[1] in edge and i[0] in edge[i[1]]:
            continue
        el += [i]
    edge_list['field']['paper']['PF_in_L1'] = el
    
    
    node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict = \
            to_torch(feature, times, edge_list, graph)
    '''
        Trace the paper_id and field_id by its own index plus the type start index
    '''
    paper_ids = np.arange(len(pids)) + node_dict['paper'][0]
    field_ids = np.arange(len(fids)) + node_dict['field'][0]
    ylabel = torch.zeros(sampn, len(cand_list))
    for x_id, p_id in enumerate(pids[:,0]):
        for f_id in pairs[p_id]:
            ylabel[x_id][cand_list.index(f_id)] = 1
    ylabel /= ylabel.sum(axis=1).view(-1, 1)
    return node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel
    
def prepare_data(pool, process_ids):
    jobs = []
    for process_id in process_ids[:-1]:
        p = pool.apply_async(pf_sample, args=(np.random.randint(2**32 - 1), train_papers, \
                                               train_pairs, train_range, batch_size))
        jobs.append(p)
    p = pool.apply_async(pf_sample, args=(np.random.randint(2**32 - 1), valid_papers, \
                                           valid_pairs, valid_range, batch_size))
    jobs.append(p)
    return jobs

In [10]:
class HANConv(MessagePassing):
    def __init__(self, in_dim, out_dim, num_relations, n_heads, dropout = 0.3, **kwargs):
        super(HANConv, self).__init__(aggr='add', **kwargs)

        self.in_dim        = in_dim
        self.out_dim       = out_dim
        self.num_relations = num_relations
        self.n_heads       = n_heads
        self.d_k           = out_dim // n_heads
        
        self.aggregat_rws   = nn.ModuleList()
        self.interact_rws   = nn.ModuleList()
        for t in range(num_relations):
            self.interact_rws.append(nn.Linear(2 * self.d_k,   1))
            self.aggregat_rws.append(nn.Linear(in_dim,   out_dim))
        self.drop            = nn.Dropout(dropout)
        self.semantic_q      = nn.Linear(out_dim, out_dim)
        self.semantic_v      = nn.Linear(out_dim, 1)
    def forward(self, node_inp, edge_index, edge_type):
        return self.propagate(edge_index, node_inp=node_inp, edge_type=edge_type)

    def message(self, edge_index_i, node_inp_i, node_inp_j, edge_type, num_nodes):
        '''
            i: target; j: source
        '''
        data_size = edge_index_i.size(0)
        res     = torch.zeros(data_size, self.n_heads, self.d_k).to(node_inp_i.device)
        res_att     = torch.zeros(data_size, self.n_heads).to(node_inp_i.device)
        res_sem     = torch.zeros(self.num_relations, 1).to(node_inp_i.device)

        for relation_id in range(self.num_relations):
            idx = (edge_type == relation_id)
            if idx.sum() == 0:
                continue
            _node_inp_i = self.aggregat_rws[relation_id](node_inp_i[idx]).view(-1, self.n_heads, self.d_k)
            _node_inp_j = self.aggregat_rws[relation_id](node_inp_j[idx]).view(-1, self.n_heads, self.d_k)
            
            res[idx] = _node_inp_j
            s = torch.cat([_node_inp_i, _node_inp_j], dim=-1)
            res_att[idx]   = self.interact_rws[relation_id](s).squeeze()
        res_att = F.leaky_relu(res_att, 0.2)
        res_att = softmax(res_att, edge_index_i, data_size)
        res = (res * res_att.view(-1, self.n_heads, 1)).view(-1, self.out_dim)
        for relation_id in range(self.num_relations):
            idx = (edge_type == relation_id)
            if idx.sum() == 0:
                continue
            sem_res = res[idx].mean(dim=0)
            sem_att = self.semantic_v(torch.tanh(self.semantic_q(sem_res)))
            res_sem[relation_id] = sem_att
        res_sem = F.softmax(res_sem, dim=0)
        res = res.view(-1, self.n_heads, self.d_k)
        ores     = torch.zeros(data_size, self.n_heads, self.d_k).to(node_inp_i.device)
        for relation_id in range(self.num_relations):
            idx = (edge_type == relation_id)
            if idx.sum() == 0:
                continue
            ores[idx] = res[idx] * res_sem[relation_id]
        del res, res_att, res_sem
        return ores.view(-1, self.out_dim)
    
    def update(self, aggr_out):
        return aggr_out

    def __repr__(self):
        return '{}(in_dim={}, out_dim={}, num_rel={})'.format(
            self.__class__.__name__, self.in_dim, self.out_dim, self.num_relations)

In [11]:
class GNN(nn.Module):
    def __init__(self, in_dim, n_hid, num_types, num_relations, n_layers, n_heads, dropout = 0.3):
        super(GNN, self).__init__()
        self.gcs = nn.ModuleList()
        self.num_types = num_types
        self.in_dim    = in_dim
        self.n_hid     = n_hid
        self.aggregat_ws   = nn.ModuleList()
        self.drop          = nn.Dropout(dropout)
        for t in range(num_types):
            self.aggregat_ws.append(nn.Linear(in_dim, n_hid))
        for l in range(n_layers):
            self.gcs.append(HANConv(n_hid, n_hid, num_relations, n_heads))
    def forward(self, node_feature, node_type, edge_index, edge_type):
        res = torch.zeros(node_feature.size(0), self.n_hid).to(node_feature.device)
        for t_id in range(self.num_types):
            aggregat_w = self.aggregat_ws[t_id]
            idx = (node_type == t_id)
            if idx.sum() == 0:
                continue
            res[idx] = torch.tanh(aggregat_w(node_feature[idx]))
        meta_xs = self.drop(res)
        del res
        for gc in self.gcs:
            meta_xs = self.drop(F.relu(gc(meta_xs, edge_index, edge_type)))
        return meta_xs

In [17]:
gnn.n_layers

AttributeError: 'GNN' object has no attribute 'n_layers'

In [7]:
'''
Paper-Field
'''
paper_ser = {}

train_pairs = {}
valid_pairs = {}
test_pairs  = {}

train_papers = {_time: {} for _time in train_range}
valid_papers = {_time: {} for _time in valid_range}
test_papers  = {_time: {} for _time in test_range}

for f_id in graph.edge_list['field']['paper']['PF_in_L1']:
    for p_id in graph.edge_list['field']['paper']['PF_in_L1'][f_id]:
        _time = graph.edge_list['field']['paper']['PF_in_L1'][f_id][p_id]
        if _time in train_range:
            if p_id not in train_pairs:
                train_pairs[p_id] = []
            train_pairs[p_id] += [f_id]
            train_papers[_time][p_id] = True
        elif _time in valid_range:
            if p_id not in valid_pairs:
                valid_pairs[p_id] = []
            valid_pairs[p_id] += [f_id]
            valid_papers[_time][p_id] = True
        else:
            if p_id not in test_pairs:
                test_pairs[p_id] = []
            test_pairs[p_id] += [f_id]
            test_papers[_time][p_id] = True
for _time in list(train_papers.keys()):
    if len(train_papers[_time]) < batch_size // 2:
        train_papers.pop(_time)
    else:
        train_papers[_time] = np.array(list(train_papers[_time].keys()))
for _time in list(valid_papers.keys()):
    if len(valid_papers[_time]) < batch_size // 2:
        valid_papers.pop(_time)
    else:
        valid_papers[_time] = np.array(list(valid_papers[_time].keys()))
for _time in list(test_papers.keys()):
    if len(test_papers[_time]) < batch_size // 2:
        test_papers.pop(_time)
    else:
        test_papers[_time] = np.array(list(test_papers[_time].keys()))

In [12]:
types = graph.get_types()
cand_list = [k for k in graph.edge_list['field']['paper']['PF_in_L1'] if len(graph.edge_list['field']['paper']['PF_in_L1'][k]) > 0]
gnn = GNN(in_dim = len(graph.node_feature['paper']['emb'][0]) + 401, n_hid = 256, num_types = len(types), \
          num_relations = len(graph.get_meta_graph()) + 1, n_heads = 8, n_layers = 3).to(device)
# gnn = torch.load('../pre-train/save/cpc_model.pt').to(device)
classifier = Classifier(256, len(cand_list)).to(device)
model = nn.Sequential(gnn, classifier)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 1000, eta_min=1e-6)

In [13]:
stats = []
pool = mp.Pool(4)
process_ids = np.arange(batch_num // 4)
st = time.time()
jobs = prepare_data(pool, process_ids)
train_step = 1500
best_val   = 0
res = []
criterion = nn.KLDivLoss(reduction='batchmean')
for epoch in np.arange(epoch_num)+1:
    '''
        Prepare Training and Validation Data
    '''
    train_data = [job.get() for job in jobs[:-1]]
    valid_data = jobs[-1].get()
    pool.close()
    pool.join()
    pool = mp.Pool(4)
    jobs = prepare_data(pool, process_ids)
    et = time.time()
    print('Data Preparation: %.1fs' % (et - st))
    
    model.train()
    train_losses = []
    torch.cuda.empty_cache()
    for batch in np.arange(2):
        for node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel in train_data:
            node_rep = gnn.forward(node_feature.to(device), node_type.to(device), edge_index.to(device), edge_type.to(device))
            res  = classifier.forward(node_rep[paper_ids])
            loss = criterion(res, ylabel.to(device))
            optimizer.zero_grad() 
            torch.cuda.empty_cache()
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.2)
            optimizer.step()
            train_losses += [loss.cpu().detach().tolist()]
            train_step += 1
            scheduler.step(train_step)
            del res, loss
    '''
        Valid
    '''
    model.eval()
    with torch.no_grad():
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = valid_data
        node_rep = gnn.forward(node_feature.to(device), node_type.to(device), edge_index.to(device), edge_type.to(device))
        res  = classifier.forward(node_rep[paper_ids])
        loss = criterion(res, ylabel.to(device))
        valid_res = []

        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            valid_res += [ai[bi].tolist()]
        valid_ndcg = np.average([ndcg_at_k(resi, len(resi)) for resi in valid_res])
        if valid_ndcg > best_val:
            best_val = valid_ndcg
            torch.save(model, './save/rgt_1.pt')
        st = time.time()
        print(("Epoch: %d (%.1fs)  LR: %.5f Train Loss: %.2f  Valid Loss: %.2f  Valid NDCG: %.4f") % \
              (epoch, (st-et), optimizer.param_groups[0]['lr'], np.average(train_losses), loss.cpu().detach().tolist(),\
              valid_ndcg))
        stats += [[np.average(train_losses), loss.cpu().detach().tolist()]]
        del res, loss
        if epoch % 5 == 0:
            '''
                Test
            '''
            _time = np.random.choice(list(test_papers.keys()))
            node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                           test_pairs, test_range, batch_size, test=True)
            paper_rep = gnn.forward(node_feature.to(device), node_type.to(device), edge_index.to(device), edge_type.to(device))[paper_ids]
            res  = classifier.forward(paper_rep)
            test_res = []
            for ai, bi in zip(ylabel, res.argsort(descending = True)):
                test_res += [ai[bi].tolist()]
            test_ndcg = np.average([ndcg_at_k(resi, len(resi)) for resi in test_res])
            print(test_ndcg)
            del res
    del train_data, valid_data

Data Preparation: 267.3s


/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type HANConv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 1 (324.2s)  LR: 0.00060 Train Loss: 4.92  Valid Loss: 3.64  Valid NDCG: 0.4868
Data Preparation: 5.9s


/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type GNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/ziniu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type HANConv. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch: 2 (320.5s)  LR: 0.00069 Train Loss: 3.45  Valid Loss: 5.37  Valid NDCG: 0.4989
Data Preparation: 6.4s
Epoch: 3 (311.7s)  LR: 0.00078 Train Loss: 2.50  Valid Loss: 3.39  Valid NDCG: 0.4902
Data Preparation: 6.2s


...

Epoch: 68 (297.1s)  LR: 0.00081 Train Loss: 1.86  Valid Loss: 2.58  Valid NDCG: 0.5822
Data Preparation: 5.9s
Epoch: 69 (294.6s)  LR: 0.00088 Train Loss: 1.93  Valid Loss: 2.99  Valid NDCG: 0.4978
Data Preparation: 5.9s
Epoch: 70 (292.7s)  LR: 0.00094 Train Loss: 1.92  Valid Loss: 2.92  Valid NDCG: 0.4718
0.5426795047502705
Data Preparation: 25.6s
Epoch: 71 (291.3s)  LR: 0.00098 Train Loss: 2.01  Valid Loss: 2.96  Valid NDCG: 0.5559
Data Preparation: 6.0s
Epoch: 72 (288.5s)  LR: 0.00100 Train Loss: 1.93  Valid Loss: 2.72  Valid NDCG: 0.5606
Data Preparation: 6.1s
Epoch: 73 (293.4s)  LR: 0.00100 Train Loss: 1.85  Valid Loss: 2.88  Valid NDCG: 0.5488
Data Preparation: 6.0s
Epoch: 74 (296.6s)  LR: 0.00098 Train Loss: 1.84  Valid Loss: 2.64  Valid NDCG: 0.5577
Data Preparation: 6.0s
Epoch: 75 (299.3s)  LR: 0.00095 Train Loss: 1.81  Valid Loss: 2.80  Valid NDCG: 0.5383
0.5154148557714577
Data Preparation: 25.7s
Epoch: 76 (305.2s)  LR: 0.00089 Train Loss: 1.77  Valid Loss: 3.15  Valid NDCG: 

Data Preparation: 6.4s
Epoch: 140 (292.8s)  LR: 0.00092 Train Loss: 1.65  Valid Loss: 2.80  Valid NDCG: 0.5281
0.5366936586141382
Data Preparation: 26.6s
Epoch: 141 (300.5s)  LR: 0.00086 Train Loss: 1.57  Valid Loss: 2.80  Valid NDCG: 0.5322
Data Preparation: 6.3s
Epoch: 142 (300.1s)  LR: 0.00079 Train Loss: 1.58  Valid Loss: 2.98  Valid NDCG: 0.5025
Data Preparation: 6.1s
Epoch: 143 (305.7s)  LR: 0.00070 Train Loss: 1.58  Valid Loss: 2.86  Valid NDCG: 0.5514
Data Preparation: 6.4s
Epoch: 144 (296.4s)  LR: 0.00061 Train Loss: 1.57  Valid Loss: 2.89  Valid NDCG: 0.5134
Data Preparation: 6.5s
Epoch: 145 (299.8s)  LR: 0.00052 Train Loss: 1.47  Valid Loss: 2.93  Valid NDCG: 0.5416
0.5690174800342487
Data Preparation: 26.3s
Epoch: 146 (287.8s)  LR: 0.00042 Train Loss: 1.57  Valid Loss: 2.79  Valid NDCG: 0.5389
Data Preparation: 6.4s
Epoch: 147 (305.5s)  LR: 0.00033 Train Loss: 1.52  Valid Loss: 2.82  Valid NDCG: 0.5593
Data Preparation: 6.5s
Epoch: 148 (299.6s)  LR: 0.00024 Train Loss: 1.52

Process ForkPoolWorker-618:
Process ForkPoolWorker-617:
Process ForkPoolWorker-619:


KeyboardInterrupt: 

Process ForkPoolWorker-620:
Traceback (most recent call last):
  File "/home/ziniu/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ziniu/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ziniu/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ziniu/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ziniu/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ziniu/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/home/ziniu/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args,

In [ ]:
stats = np.array(stats)
plt.plot(stats[:,0])
plt.plot(stats[:,1])
plt.show()

In [ ]:
model.eval()
gnn, classifier = model
with torch.no_grad():
    test_res = []
    for _ in range(10):
        _time = np.random.choice(list(test_papers.keys()))
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                       test_pairs, test_range, batch_size, test=True)
        paper_rep = gnn.forward(node_feature.to(device), node_type.to(device), edge_index.to(device))[paper_ids]
        res = classifier.forward(paper_rep)
        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            test_res += [ai[bi].tolist()]
    test_ndcg = [ndcg_at_k(resi, len(resi)) for resi in test_res]
    print(np.average(test_ndcg), np.var(test_ndcg))
    test_mrr = mean_reciprocal_rank(test_res)
    print(np.average(test_mrr), np.var(test_mrr))

In [ ]:
best_model = torch.load('./save/rgt_1.pt')

In [ ]:
best_model.eval()
gnn, classifier = best_model
with torch.no_grad():
    test_res = []
    for _ in range(10):
        _time = np.random.choice(list(test_papers.keys()))
        node_feature, node_type, edge_time, edge_index, edge_type, field_ids, paper_ids, ylabel = pf_sample(np.random.randint(2 ** 32 - 1), test_papers, \
                                                       test_pairs, test_range, batch_size, test=True)
        paper_rep = gnn.forward(node_feature.to(device), node_type.to(device), edge_index.to(device))[paper_ids]
        res = classifier.forward(paper_rep)
        for ai, bi in zip(ylabel, res.argsort(descending = True)):
            test_res += [ai[bi].tolist()]
    test_ndcg = [ndcg_at_k(resi, len(resi)) for resi in test_res]
    print(np.average(test_ndcg), np.var(test_ndcg))
    test_mrr = mean_reciprocal_rank(test_res)
    print(np.average(test_mrr), np.var(test_mrr))